In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import re
import time
import numpy as np

In [ ]:
ano = '202223'
url = 'https://fbref.com/pt/comps/32/2022-2023/cronograma/2022-2023-Primeira-Liga-Resultados-e-Calendarios'
playoff = False

In [ ]:
df_update = pd.read_html(url, extract_links="all")
df_update = df_update[0]

df_update.columns = [col[0] for col in df_update.columns]

df_update_getSecondTuple = ['Relatório da Partida']

for column in df_update.columns:
    if column not in df_update_getSecondTuple:
        df_update[column] = df_update[column].apply(lambda x: x[0])
    else:
        df_update[column] = df_update[column].apply(lambda x: x[1])
        
if playoff:
    df_update = df_update[:-2]
    df_update.drop(['Rodada'],inplace=True)
            
df_update['goals_home'] = df_update['Resultado'].str.split('–').str[0]
df_update['goals_away'] = df_update['Resultado'].str.split('–').str[1]
df_update.drop(['Resultado','Notas'],axis=1,inplace=True)
values = ['None']
df_update = df_update[df_update['Relatório da Partida'].isna() == False].reset_index(drop=True) 
df_update.rename({'xG': 'xG_home', '(': 'xG_away', 'Em casa': 'home', 'Visitante': 'away', 'Público': 'Assistance', 'Sem': 'Jornada'}, axis=1, inplace=True)

In [ ]:
df_update

In [ ]:
df

In [ ]:
try:
    df = pd.read_csv(f'dataset_{ano}.csv')
    df.drop('Unnamed: 0',inplace=True)
except:
    df = df_update.copy()
    
df_update['Assistance'] =  df_update['Assistance'].str.replace('.', '').replace('', 0)
df_update['Assistance'] = df_update['Assistance'].astype(int)
df['Assistance'] = df_update['Assistance']

for index, row in df[df['Relatório da Partida'].str.startswith('/pt/stathead/')].iterrows():
    df.loc[index] = df_update.loc[index]

for index, row in df.iterrows():
    
    if row['Relatório da Partida'].startswith('/pt/partidas/') and ('jogador1_home' not in df.columns or pd.isna(df.loc[index,'jogador1_home'])):
    
        start = time.time()

        url = row['Relatório da Partida']
        print(url)
        r = requests.get('http://fbref.com' + url)
        soup = BeautifulSoup(r.content, 'html.parser')
        tables = soup.find_all('table')
        try:
            table = tables[0]
            table2 = tables[1] 
        except Exception:
            indice = index
        
        home_players = pd.read_html(str(table))[0].drop(11)
        away_players = pd.read_html(str(table2))[0].drop(11)
        df.loc[index,'formacao_home'] = home_players.columns[0].split('(')[1][:-1]
        df.loc[index,'formacao_away'] = away_players.columns[0].split('(')[1][:-1]
        home_players = home_players.transpose().reset_index().drop('index',axis=1)
        away_players = away_players.transpose().reset_index().drop('index',axis=1)

        print(f'meio: {time.time() - start}')
        
        stats_div = soup.find('div', {'id': 'team_stats_extra'})

        # extrai o conteúdo da div
        stats_text = stats_div.text

        linhas = stats_text.split('\n')
        palavras_chave = ['Faltas', 'Escanteios', 'Cruzamentos', 'Contatos', 'Bote defensivo', 'Cortes', 'Jogadas aéreas', 'Defesas', 'Impedimentos', 'Tiro de meta', 'Cobrança de lateral', 'Bolas longas']

        lista_strings = [string for string in linhas for palavra in palavras_chave if palavra in string]

        nova_lista = []
        for string in lista_strings:
            match = re.match(r'^(\d+)([a-zA-Z\s]+)(\d+)$', string)
            if match:
                nova_lista.append([int(match.group(1)), match.group(2).strip(), int(match.group(3))])
        
        for x in nova_lista:
            df.loc[index, f"{x[1]}_home"] = x[0]
            df.loc[index, f"{x[1]}_away"] = x[2]
            
            
        cards = {
            'home_team': {
                'yellow_card': 0,
                'red_card': 0,
                'yellow_red_card': 0
            },
            'away_team': {
                'yellow_card': 0,
                'red_card': 0,
                'yellow_red_card': 0
            }
        }

        home_team_cards_div = soup.find_all('div', class_='cards')[0]
        home_team_cards_spans = home_team_cards_div.find_all('span')
        for span in home_team_cards_spans:
            cards['home_team'][span['class'][0]] += 1

        away_team_cards_div = soup.find_all('div', class_='cards')[1]
        away_team_cards_spans = away_team_cards_div.find_all('span')
        for span in away_team_cards_spans:
            cards['away_team'][span['class'][0]] += 1
        
        df.loc[index, 'yellow_card_home'] = cards['home_team']['yellow_card']
        df.loc[index, 'red_card_home'] = cards['home_team']['red_card']
        df.loc[index, 'yellow_red_card_home'] = cards['home_team']['yellow_red_card']
        df.loc[index, 'yellow_card_away'] = cards['away_team']['yellow_card']
        df.loc[index, 'red_card_away'] = cards['away_team']['red_card']
        df.loc[index, 'yellow_red_card_away'] = cards['away_team']['yellow_red_card']
        
        for i in range(len(home_players.columns) - 1):
            column_name = f'jogador{i+1}_home'
            df.loc[index, column_name] = home_players.iloc[1, i]
        for i in range(len(away_players.columns) - 1):
            column_name = f'jogador{i+1}_away'
            df.loc[index, column_name] = away_players.iloc[1, i]

        print(f'fim: {time.time() - start}')
        if time.time() - start < 6.0:
            time.sleep(6.1 - (time.time() - start))
            
df.to_csv(f'dataset_{ano}.csv')
df.head()

In [ ]:
df.to_csv(f'dataset_{ano}.csv')
df

In [ ]:
### 538
df_538 = pd.read_csv('https://projects.fivethirtyeight.com/soccer-api/club/spi_matches.csv')
df_538 = df_538[df_538['league'] == 'Portuguese Liga']
df_538 = df_538[df_538['season'] == 2022] 


In [ ]:
df_538['league'].unique()

In [ ]:
df_update = pd.read_html('https://fbref.com/pt/comps/32/cronograma/Primeira-Liga-Resultados-e-Calendarios#sched_2022-2023_32_1', extract_links="all")
df_update = df_update[0]

In [ ]:
import numpy as np
df = pd.read_csv('output_dataset_exemplo.csv')
df.set_index('Unnamed: 0', inplace=True)
df_update['Assistance'] =  df_update['Assistance'].str.replace('.', '').replace('', np.nan)
df_update['Assistance'] = df_update['Assistance'].astype(float)
df['Assistance'] = df_update['Assistance']

In [ ]:
df_update.columns = [col[0] for col in df_update.columns]

df_update_getSecondTuple = ['Relatório da Partida']

for column in df_update.columns:
    if column not in df_update_getSecondTuple:
        df_update[column] = df_update[column].apply(lambda x: x[0])
    else:
        df_update[column] = df_update[column].apply(lambda x: x[1])

In [ ]:
df2 = df_update[df_update['Assistance'].isna() == False]#.reset_index() 
#

In [ ]:
df

In [ ]:
df_diff = df.compare(df_update)

# Seleciona as linhas que são diferentes em df e df_update
row_idx = df_diff.index[df_diff.ne("").any(axis=1)]

# Substitui as linhas em df pelos valores atualizados em df_update
#df.loc[row_idx] = df_update.loc[row_idx]